### 1. SSD활용 얼굴검출 예제

In [3]:
import os
import cv2

model, config = "models/res10_300x300_ssd_iter_140000_fp16.caffemodel", "models/deploy.prototxt"
cap = cv2.VideoCapture(0)   #카메라 오픈
net = cv2.dnn.readNet(model, config)  #모델 불러오기
while True:
    ret, frame = cap.read()   #카메라 프레임 읽어오기
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123)) #이미지 크기, 컬러 평균값 지정
    net.setInput(blob) #입력 설정
    out = net.forward() #추론
    detect = out[0, 0, :, :]   # 출력 배열에서 뒷쪽 2개 차원만 활용  [200,7]
    (h, w) = frame.shape[:2]  #카메라 프레임의 크기 읽어오기 
    for i in range(detect.shape[0]):     #200행을 차례로 불러오기
        confidence = detect[i, 2]          # c 값을 읽어 confidence에 저장
        if confidence < 0.5:  #confidence 가 0.5보다 작을때는 skip 
            break
        #현재 프레임에 맞춰서 좌표 계산
        x1,  y1, x2, y2 = int(detect[i, 3] * w), int(detect[i, 4] * h), int(detect[i, 5] * w), int(detect[i, 6] * h)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0))  # 박스 그리기
        label = f"Face: {confidence:4.2f}"    #확률값 출력 
        cv2.putText(frame, label, (x1, y1 - 1), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1, cv2.LINE_AA)
    cv2.imshow("frame", frame)  #출력
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()


### 2. YOLO활용 사물검출 예제

In [ ]:
import cv2
import sys
import numpy as np

# 모델 & 설정 파일
model, config, class_labels = "models/yolov3.weights", "models/yolov3.cfg", "models/coco.names"
cap = cv2.VideoCapture(0)   #카메라 오픈
net = cv2.dnn.readNet(model, config) #모델 불러오기

classes = [] # 클래스 이름 불러오기
with open(class_labels, "rt") as f:
    classes = f.read().rstrip("\n").split("\n")

colors = np.random.uniform(0, 255, size=(len(classes), 3)) #80개의 클래스에 랜덤한 바운딩 박스 컬러 부여
layer_names = net.getLayerNames() # 출력 레이어 이름 받아오기, 82번, 94번, 106번 layer 사용
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
confThreshold, nmsThreshold = 0.5, 0.4

while True:
    ret, frame = cap.read()   #카메라 프레임 읽어오기
    blob = cv2.dnn.blobFromImage(frame, 1/255., (320, 320), swapRB=True) #이미지 크기, 컬러 평균값 지정
    net.setInput(blob) #입력설정
    outs = net.forward(output_layers) #추론
    h, w = frame.shape[:2]
    class_ids, confidences, boxes = [], [], []

    for out in outs:
        for detection in out:
            scores = detection[5:]     # score에 5~90열 저장, 
            class_id = np.argmax(scores)   #확률값이 가장 높은 열 번호 추출
            confidence = scores[class_id]   # 확률값 저장
            if confidence > confThreshold:
                # 바운딩 박스 중심 좌표 & 박스 크기 구하기 #바운딩 박스 좌표값이 0~1 이므로 이미지 사이즈 곱함
                cx, cy, bw, bh = int(detection[0] * w), int(detection[1] * h), int(detection[2] * w), int(detection[3] * h)
                sx, sy = int(cx - bw / 2), int(cy - bh / 2) # 바운딩 박스 좌상단 좌표 구하기
                boxes.append([sx, sy, bw, bh])
                confidences.append(float(confidence))
                class_ids.append(int(class_id))

    # 비최대 억제; condidences가 가장 높은 boxes 선택
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

    for i in indices:    # 이미지에 bounding box와 class, confidence 디스플레이
        i = i[0]
        sx, sy, bw, bh = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2}"
        color = colors[class_ids[i]]
        cv2.rectangle(frame, (sx, sy, bw, bh), color, 2)
        cv2.putText(frame, label, (sx, sy - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

    cv2.imshow("frame", frame)  
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()